In [27]:
import json

def get_first_recipe(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
        first_recipe = data[0]
        return first_recipe

def parse_quantity(quantity_str):
    quantity_str = quantity_str.lower()
    quantity = 0
    if 'g' in quantity_str:  # grams
        quantity = float(quantity_str.split()[0])
    elif 'verres' in quantity_str:  # glasses
        quantity = float(quantity_str.split()[0]) * 200
    elif 'petite tasse' in quantity_str:  # small cup
        quantity = float(quantity_str.split()[0]) * 100
    elif quantity_str.isdigit():  # plain number
        quantity = float(quantity_str)
    return quantity

def get_ingredient_costs(filename, recipe):
    with open(filename, 'r') as f:
        data = json.load(f)
        ingredient_names = [ingredient['name'] for ingredient in recipe['ingredients']] 
        ingredient_names = list(dict.fromkeys(ingredient_names))
        total_costs = {'USP': {'direct_price': 0, 'quantity_price': 0, 'kg_price': 0}, 
                       'ALDI': {'direct_price': 0, 'quantity_price': 0, 'kg_price': 0}}
        for ingredient_name in ingredient_names:
            quantity = next(ingredient['quantity'] for ingredient in recipe['ingredients'] if ingredient['name'] == ingredient_name) 
            quantity = parse_quantity(quantity)
            for source in ['USP', 'ALDI']:
                for item in data:
                    if item['ingredient'] == ingredient_name and item['source'] == source or ingredient_name in item.get('other_ingredients', []) and item['source'] == source:
                        item['price_per_kg'] = item['price_kg']
                        item['direct_price'] = item['price']
                        if item['price_kg'] != -1: 
                            if quantity == 0:  # Set a default quantity if none 
                                quantity = 0.1
                            item['quantity_price'] = quantity * item['price_kg']

                        print(f"Source : {item['source']}")
                        print(f"Ingrédient : {item['ingredient']}")
                        print(f"Prix par kg : {item['price_per_kg']}")
                        print(f"Prix direct : {item['direct_price']}")
                        print(f"Prix selon quantité : {item.get('quantity_price', 'Non disponible')}")
                        print("-----------------------------")
                        
                        # add prices 
                        total_costs[source]['kg_price'] += item['price_per_kg']
                        total_costs[source]['direct_price'] += item['direct_price']
                        total_costs[source]['quantity_price'] += item.get('quantity_price', 0)
                        break


    for source in total_costs.keys():
        print(f"\nTotal cost for {source} ingredients:")
        print(f"Price per kg: {round(total_costs[source]['kg_price'],2)}")
        print(f"Direct price: {round(total_costs[source]['direct_price'],2)}")
        print(f"Price according to quantity: {round(total_costs[source]['quantity_price'],2)}")
        

        print("-----------------------------")


recipe_file = 'recipe_marmiton.json'
ingredient_file = 'items_ingredient.json'

recipe = get_first_recipe(recipe_file)
get_ingredient_costs(ingredient_file, recipe)


Source : USP
Ingrédient : tomate
Prix par kg : 5.800000000000001
Prix direct : -1
Prix selon quantité : 0.5800000000000001
-----------------------------
Source : ALDI
Ingrédient : tomate
Prix par kg : 3.0
Prix direct : 1.19
Prix selon quantité : 0.30000000000000004
-----------------------------
Source : USP
Ingrédient : lait
Prix par kg : 1.25
Prix direct : -1
Prix selon quantité : 0.125
-----------------------------
Source : ALDI
Ingrédient : lait
Prix par kg : -1
Prix direct : 2.99
Prix selon quantité : Non disponible
-----------------------------
Source : USP
Ingrédient : oeuf
Prix par kg : 0.6499999999999999
Prix direct : -1
Prix selon quantité : 2.5999999999999996
-----------------------------
Source : ALDI
Ingrédient : gruyère
Prix par kg : 18.3
Prix direct : 3.65
Prix selon quantité : 1.83
-----------------------------
Source : ALDI
Ingrédient : olives
Prix par kg : 10.0
Prix direct : 1.99
Prix selon quantité : 1.0
-----------------------------
Source : ALDI
Ingrédient : jambon
